In [ ]:
%pip install pandas
%pip install transformers
%pip install accelerate

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [31]:
import pandas as pd
df = pd.read_csv('test_subset.csv')


In [4]:
texts = list(df.text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

prompt = f"Classify the sentiment of each text into ONE of the three classes: neutral, negative or positive. Split the answer in a dictionary with two keys:Text and Label. Text: "

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    print(answer)
print(len(answers))

In [ ]:
labels = ['negative','neutral','positive']
predicted_labels = []
for answer in answers:
    label = answer.split('[/INST]')[1].split(':')[2].split('}')[0].lower().strip(' "" ').strip().strip(' " ')
    print(label)
    if label in labels:
        predicted_labels.append(label)
    else:
        predicted_labels.append('NA')
len(predicted_labels)


In [33]:
df['predicted_labels'] = predicted_labels
df = df[df['predicted_labels'] != 'NA']
df = df.reset_index(drop=True)
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans,predicted_labels
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...,negative
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel,neutral
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...,negative
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...,negative
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']",neutral
...,...,...,...,...,...,...,...,...,...
262,ekaterini@gmail.com,1219,Mit dem Rassisten und Ultra nationalen Netanya...,negative,Netanyahu,"['Rassisten', 'keine Verhandlungen mehr möglic...",...,...,negative
263,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...,negative
264,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...,neutral
265,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...,positive


In [3]:
correct_predictions = 0
for i in range(len(df)):
    if df['sentiment'][i] == df['predicted_labels'][i]:
        correct_predictions += 1
print('correct predictions:',correct_predictions)

correct predictions: 205


In [4]:
Accuracy = (correct_predictions/len(df))*100
print('Accuracy of mistral before fine tuning is',Accuracy, '%')

Accuracy of mistral before fine tuning is 76.77902621722846 %


In [36]:
df.to_csv('ResultsBeforeFineTuning_SequnceClassification_mistral.csv', index=False)